## Audio

In [1]:
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import pandas as pd
import wave
import librosa
import re
from tensorflow_hub import load, Module
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input, LSTM, Bidirectional, RNN,  GRU, Conv1D, MaxPooling1D
from tensorflow.keras.models import Model
from keras.utils import to_categorical
from tensorflow.keras.layers import Concatenate
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error, mean_squared_error
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import contractions
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop, Adam, Adadelta
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import noisereduce as nr
%matplotlib inline

In [3]:
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/data_ml5/

Mounted at /content/drive
/content/drive/MyDrive/data_ml5


In [19]:
train_split_df = pd.read_csv('train_split_Depression_AVEC2017.csv')
test_split_df = pd.read_csv('dev_split_Depression_AVEC2017.csv')
train_split_num = train_split_df[['Participant_ID']]['Participant_ID'].tolist()
test_split_num = test_split_df[['Participant_ID']]['Participant_ID'].tolist()
# train_split_clabel = train_split_df[['PHQ8_Binary']]['PHQ8_Binary'].tolist()
# test_split_clabel = test_split_df[['PHQ8_Binary']]['PHQ8_Binary'].tolist()
train_split_clabel = train_split_df[['PHQ8_Score']]['PHQ8_Score'].tolist()
test_split_clabel = test_split_df[['PHQ8_Score']]['PHQ8_Score'].tolist()

In [ ]:
def extract_features(number, audio_features, target, audio_targets, mode):
    transcript = pd.read_csv('{0}_TRANSCRIPT.csv'.format(number), sep='\t').fillna('')

    wavefile = wave.open('{0}_AUDIO.wav'.format(number, 'r'))
    sample_rate = wavefile.getframerate()
    nframes = wavefile.getnframes()
    wave_data = np.frombuffer(wavefile.readframes(nframes), dtype=np.short)
    reduced_noise = nr.reduce_noise(y=wave_data, sr=sr)

    start_time = 0
    stop_time = 0

    signal = []

    global counter_train

    for t in transcript.itertuples():
        if getattr(t,'speaker') == 'Ellie':
            continue
        elif getattr(t,'speaker') == 'Participant':
            if 'scrubbed_entry' in getattr(t,'value'):
                continue
            start_time = int(getattr(t,'start_time')*sample_rate)
            stop_time = int(getattr(t,'stop_time')*sample_rate)
            signal = np.hstack((signal, reduced_noise[start_time:stop_time].astype(float)))

    clip = sample_rate*1*15
    if target == 1 and mode == 'train':
    # if target >= 10 and mode == 'train':
        times = 3 if counter_train < 48 else 2
        for i in range(times):
            if clip*(i+1) > len(signal):
                continue
            sample = signal[clip*i:clip*(i+1)]
            melspec = librosa.feature.melspectrogram(y=sample, n_mels=80,sr=sample_rate)
            audio_features.append(melspec)
            audio_targets.append(target)
            counter_train+=1
    else:
        sample = signal[:clip]
        melspec = librosa.feature.melspectrogram(y=sample, n_mels=80, sr=sample_rate)
        audio_features.append(melspec)
        audio_targets.append(target)
    print(melspec.shape)
    print('{}_P feature done'.format(number))

In [ ]:
audio_features_train = []
audio_ctargets_train = []
counter_train = 0

for index in range(len(train_split_num)):
    extract_features(train_split_num[index], audio_features_train, train_split_clabel[index], audio_ctargets_train, 'train')


In [ ]:

audio_features_test = []
audio_ctargets_test = []

for index in range(len(test_split_num)):
    extract_features(test_split_num[index], audio_features_test, test_split_clabel[index], audio_ctargets_test, 'test')

print(np.shape(audio_ctargets_train), np.shape(audio_ctargets_test))

In [ ]:
np.savez('features/train_samples_cla.npz', audio_features_train)
np.savez('features/test_samples_cla.npz', audio_features_test)
np.savez('features/train_labels_cla.npz', audio_ctargets_train)
np.savez('features/test_labels_cla.npz', audio_ctargets_test)

In [5]:
features_train = np.load('features/train_samples_cla.npz', allow_pickle=True)['arr_0']
features_test = np.load('features/test_samples_cla.npz', allow_pickle=True)['arr_0']
targets_train = np.load('features/train_labels_cla.npz', allow_pickle=True)['arr_0']
ctargets_test = np.load('features/test_labels_cla.npz', allow_pickle=True)['arr_0']

In [6]:
X_train = np.array(features_train)
Y_train = np.array(targets_train)
X_test = np.array(features_test)
Y_test = np.array(ctargets_test)

In [7]:
X_train = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_train])
X_test = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_test])

In [8]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(153, 80, 469)
(153,)
(35, 80, 469)
(35,)


In [36]:
train_y = to_categorical(Y_train)
test_y = to_categorical(Y_test)

### Classification

In [43]:
model = Sequential()

model.add(Conv1D(32, 3, activation='relu', input_shape=(80, 469)))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(2, activation = 'softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, train_y, epochs=15, batch_size=8, validation_data=(X_test, test_y))

Epoch 1/15
20/20 [==============================] - 7s 118ms/step - loss: 0.6932 - accuracy: 0.4706 - val_loss: 0.6931 - val_accuracy: 0.5429
Epoch 2/15
20/20 [==============================] - 1s 42ms/step - loss: 0.6931 - accuracy: 0.4706 - val_loss: 0.6947 - val_accuracy: 0.3429
Epoch 3/15
20/20 [==============================] - 1s 43ms/step - loss: 0.6928 - accuracy: 0.4967 - val_loss: 0.6957 - val_accuracy: 0.3429
Epoch 4/15
20/20 [==============================] - 1s 42ms/step - loss: 0.6924 - accuracy: 0.4967 - val_loss: 0.6975 - val_accuracy: 0.3429
Epoch 5/15
20/20 [==============================] - 1s 43ms/step - loss: 0.6919 - accuracy: 0.4967 - val_loss: 0.6960 - val_accuracy: 0.3429
Epoch 6/15
20/20 [==============================] - 1s 43ms/step - loss: 0.6911 - accuracy: 0.4967 - val_loss: 0.6994 - val_accuracy: 0.3429
Epoch 7/15
20/20 [==============================] - 1s 32ms/step - loss: 0.6891 - accuracy: 0.4967 - val_loss: 0.7004 - val_accuracy: 0.3429
Epoch 8/15
2

In [44]:
loss, accuracy = model.evaluate(X_test, test_y)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

y_pred = model.predict(X_test)
predicted_1 = [1 if x[1] > x[0] else 0 for x in y_pred]
print(classification_report(Y_test, predicted_1))

2/2 [==============================] - 0s 16ms/step - loss: 0.9857 - accuracy: 0.7714
Test Loss: 0.9856722354888916, Test Accuracy: 0.7714285850524902
2/2 [==============================] - 1s 9ms/step
              precision    recall  f1-score   support

           0       0.89      0.74      0.81        23
           1       0.62      0.83      0.71        12

    accuracy                           0.77        35
   macro avg       0.76      0.79      0.76        35
weighted avg       0.80      0.77      0.78        35



### Regression

In [46]:
features_train = np.load('features/train_samples_reg.npz', allow_pickle=True)['arr_0']
features_test = np.load('features/test_samples_reg.npz', allow_pickle=True)['arr_0']
targets_train = np.load('features/train_labels_reg.npz', allow_pickle=True)['arr_0']
ctargets_test = np.load('features/test_labels_reg.npz', allow_pickle=True)['arr_0']

In [167]:
X_train = np.array(features_train)
Y_train = np.array(targets_train)
X_test = np.array(features_test)
Y_test = np.array(ctargets_test)

In [177]:
model = Sequential()

model.add(Conv1D(32, 3, activation='relu', input_shape=(80, 469)))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(1, activation = 'linear'))

model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0001), metrics=['mean_absolute_error'])

model.fit(X_train,Y_train,batch_size=4,epochs=15)

Epoch 1/15
39/39 [==============================] - 6s 20ms/step - loss: 108.7297 - mean_absolute_error: 8.6373
Epoch 2/15
39/39 [==============================] - 1s 20ms/step - loss: 102.6986 - mean_absolute_error: 8.3268
Epoch 3/15
39/39 [==============================] - 1s 20ms/step - loss: 94.9049 - mean_absolute_error: 7.9490
Epoch 4/15
39/39 [==============================] - 1s 20ms/step - loss: 81.9712 - mean_absolute_error: 7.3632
Epoch 5/15
39/39 [==============================] - 1s 24ms/step - loss: 63.0287 - mean_absolute_error: 6.3814
Epoch 6/15
39/39 [==============================] - 1s 33ms/step - loss: 47.9408 - mean_absolute_error: 5.6684
Epoch 7/15
39/39 [==============================] - 1s 33ms/step - loss: 39.6455 - mean_absolute_error: 5.2848
Epoch 8/15
39/39 [==============================] - 1s 32ms/step - loss: 35.5722 - mean_absolute_error: 5.0499
Epoch 9/15
39/39 [==============================] - 1s 32ms/step - loss: 34.6597 - mean_absolute_error: 4.9925

In [175]:
loss, mean_absolute_error = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}, Test Mean Absolute Error: {mean_absolute_error}")

2/2 [==============================] - 0s 16ms/step - loss: 33.0534 - mean_absolute_error: 4.9541
Test Loss: 33.05341720581055, Test Mean Absolute Error: 4.954130172729492


In [176]:
y_pred = model.predict(X_test)
print(np.sqrt(mean_squared_error(Y_test,y_pred)))

2/2 [==============================] - 1s 12ms/step
5.749210283188846


## Text

In [ ]:
topics = []
with open('questions.txt', 'r') as f:
    for line in f.readlines():
        topics.append(line.strip('\n').strip())

In [ ]:
def identify_topics(sentence):
    sentence = re.sub(r'\(|\)', '', sentence)
    pattern = r'\b(what|how|where|when|why|are|do|have|who|who\'s|what\'s|why\'d|what\'d)\b(.*)$'
    match = re.search(pattern, sentence, re.IGNORECASE)
    if match:
        question = match.group(0).strip()
        if question in topics:
          return True
    return False

def expanding_contractions(text):
  expanded_words = []
  for word in text.split():

    expanded_words.append(contractions.fix(word))

  text = ' '.join(expanded_words)
  return text

def extract_features(number, features, target, targets, mode):
    print(number)
    transcript = pd.read_csv('{0}_TRANSCRIPT.csv'.format(number), sep='\t').fillna('')

    responses = []
    response = ''
    response_flag = False
    signal = []

    global counter_train

    for row in transcript.itertuples():
        if row.speaker == 'Ellie':

            content = row.value.strip()
            if identify_topics(content):
                response_flag = True
                if len(response) != 0:
                    response = expanding_contractions(response.strip())
                    response = re.sub(r'[^\w\s]', '', response)
                    responses.append(response)
                response = ''
            elif response_flag and len(content.split()) > 4:
                response_flag = False
                if len(response) != 0:
                    response = expanding_contractions(response)
                    response = re.sub(r'[^\w\s]', '', response)
                    responses.append(response)
                response = ''
        elif row.speaker == 'Participant':
            if 'scrubbed_entry' in row.value:
                continue
            elif response_flag:
                response +=' ' +row.value.split('\n')[0].strip()

    if len(responses) == 0:
      print("Empty")
    else:
      # if target == 1 and mode == 'train':
      if target >= 10 and mode == 'train':
        times = 3 if counter_train < 48 else 2
        for i in range(times):
              if 10*(i+1) > len(responses):
                continue
              text_f = responses[i*10:(i+1)*10]
              features.append(text_f)
              targets.append(target)
              counter_train+=1
      else:
          text_f = responses[:10]
          features.append(text_f)
          targets.append(target)

In [ ]:
counter_train = 0

text_features_train = []
text_ctargets_train = []

for i in range(len(train_split_num)):
    extract_features(train_split_num[i], text_features_train,  train_split_clabel[i], text_ctargets_train, 'train')

303
304
305
310
312
313
315
316
317
318
319
320
321
322
324
325
326
327
328
330
333
336
338
339
340
341
343
344
345
347
348
350
351
352
353
355
356
357
358
360
362
363
364
366
368
369
370
371
372
374
375
376
379
380
383
385
386
391
392
393
397
400
401
402
409
412
414
415
416
419
423
425
426
427
428
429
430
433
434
437
441
443
444
445
446
447
448
449
454
455
456
457
459
463
464
468
471
473
474
475
478
479
485
486
487
488
491


In [ ]:
text_features_test = []
text_ctargets_test = []

for i in range(len(test_split_num)):
    extract_features(test_split_num[i], text_features_test, test_split_clabel[i], text_ctargets_test, 'test')

print(np.shape(text_features_train), np.shape(text_features_test))

302
307
331
335
346
367
377
381
382
388
389
390
395
403
404
406
413
417
418
420
422
436
439
440
451
Empty
458
Empty
472
476
477
482
483
484
489
490
492
(154, 10) (33, 10)


In [ ]:
X_train = [' '.join(sublist) for sublist in text_features_train]
np.shape(X_train)

(154,)

In [ ]:
X_test = [' '.join(sublist) for sublist in text_features_test]

In [ ]:
Y_train = list(text_ctargets_train)
Y_test = list(text_ctargets_test)

In [ ]:
train_y = to_categorical(Y_train)
test_y = to_categorical(Y_test)

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

### Classification

In [ ]:
inputs = Input(name='inputs',shape=[max_len])

layer = Embedding(max_words,100,input_length=max_len)(inputs)
layer = GRU(64, return_sequences=True)(layer)
layer = Dropout(0.3)(layer)
layer = GRU(128, return_sequences=False)(layer)
layer = Dropout(0.3)(layer)
layer = Dense(2,name='out_layer')(layer)
layer = Activation('softmax')(layer)
model = Model(inputs=inputs,outputs=layer)

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
model.fit(sequences_matrix,train_y,batch_size=4,epochs=10, validation_data=(test_sequences_matrix, test_y))

Epoch 1/10
39/39 [==============================] - 13s 172ms/step - loss: 0.6927 - accuracy: 0.5294 - val_loss: 0.6910 - val_accuracy: 0.6667
Epoch 2/10
39/39 [==============================] - 7s 188ms/step - loss: 0.6916 - accuracy: 0.5556 - val_loss: 0.6912 - val_accuracy: 0.5455
Epoch 3/10
39/39 [==============================] - 7s 183ms/step - loss: 0.6892 - accuracy: 0.6144 - val_loss: 0.6914 - val_accuracy: 0.5758
Epoch 4/10
39/39 [==============================] - 6s 156ms/step - loss: 0.6873 - accuracy: 0.6405 - val_loss: 0.6876 - val_accuracy: 0.6667
Epoch 5/10
39/39 [==============================] - 8s 210ms/step - loss: 0.6824 - accuracy: 0.6601 - val_loss: 0.6856 - val_accuracy: 0.6970
Epoch 6/10
39/39 [==============================] - 5s 141ms/step - loss: 0.6756 - accuracy: 0.7712 - val_loss: 0.6820 - val_accuracy: 0.7273
Epoch 7/10
39/39 [==============================] - 6s 165ms/step - loss: 0.6585 - accuracy: 0.7974 - val_loss: 0.6693 - val_accuracy: 0.7273
Epoch

In [ ]:
y_pred = model.predict(test_sequences_matrix)
predicted_1 = [1 if x[1] > x[0] else 0 for x in y_pred]
print(classification_report(Y_test, predicted_1))

2/2 [==============================] - 3s 59ms/step
              precision    recall  f1-score   support

           0       0.85      0.81      0.83        21
           1       0.69      0.75      0.72        12

    accuracy                           0.79        33
   macro avg       0.77      0.78      0.77        33
weighted avg       0.79      0.79      0.79        33



### Regression

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
inputs = Input(name='inputs',shape=[max_len])

layer = Embedding(max_words,100,input_length=max_len)(inputs)
layer = GRU(64, return_sequences=True)(layer)
layer = Dropout(0.3)(layer)
layer = GRU(128, return_sequences=False)(layer)
layer = Dropout(0.3)(layer)
layer = Dense(1,name='out_layer')(layer)
layer = Activation('linear')(layer)
model = Model(inputs=inputs,outputs=layer)

model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0001), metrics=['mean_absolute_error'])

model.fit(sequences_matrix,Y_train,batch_size=4,epochs=10, validation_data=(test_sequences_matrix, Y_test))

Epoch 1/10
39/39 [==============================] - 12s 172ms/step - loss: 105.9960 - mean_absolute_error: 8.4830 - val_loss: 79.6350 - val_mean_absolute_error: 6.8398
Epoch 2/10
39/39 [==============================] - 9s 224ms/step - loss: 101.3067 - mean_absolute_error: 8.2632 - val_loss: 71.4100 - val_mean_absolute_error: 6.3977
Epoch 3/10
39/39 [==============================] - 7s 170ms/step - loss: 53.6652 - mean_absolute_error: 5.9190 - val_loss: 43.4985 - val_mean_absolute_error: 5.5882
Epoch 4/10
39/39 [==============================] - 8s 198ms/step - loss: 37.6220 - mean_absolute_error: 5.0872 - val_loss: 35.8473 - val_mean_absolute_error: 5.2153
Epoch 5/10
39/39 [==============================] - 10s 259ms/step - loss: 33.5753 - mean_absolute_error: 4.8557 - val_loss: 36.3294 - val_mean_absolute_error: 5.2376
Epoch 6/10
39/39 [==============================] - 6s 156ms/step - loss: 34.9698 - mean_absolute_error: 5.0040 - val_loss: 36.8150 - val_mean_absolute_error: 5.2577


In [ ]:
loss, mean_absolute_error = model.evaluate(test_sequences_matrix, Y_test)
print(f"Test Loss: {loss}, Test Mean Absolute Error: {mean_absolute_error}")

2/2 [==============================] - 0s 26ms/step - loss: 34.5844 - mean_absolute_error: 5.1658
Test Loss: 34.58436584472656, Test Mean Absolute Error: 5.165834903717041


In [ ]:
y_pred = model.predict(test_sequences_matrix)
print(np.sqrt(mean_squared_error(Y_test,y_pred)))

2/2 [==============================] - 1s 37ms/step
5.880847177270091


## Fusion

In [18]:
topics = []
with open('questions.txt', 'r') as f:
    for line in f.readlines():
        topics.append(line.strip('\n').strip())

In [20]:
def identify_topics(sentence):
    sentence = re.sub(r'\(|\)', '', sentence)
    pattern = r'\b(what|how|where|when|why|are|do|have|who|who\'s|what\'s|why\'d|what\'d)\b(.*)$'
    match = re.search(pattern, sentence, re.IGNORECASE)
    if match:
        question = match.group(0).strip()
        if question in topics:
          return True
    return False

def expanding_contractions(text):
  expanded_words = []
  for word in text.split():

    expanded_words.append(contractions.fix(word))

  text = ' '.join(expanded_words)
  return text

def extract_features(number, text_features, audio_features, target, targets, mode):
    print(number)
    transcript = pd.read_csv('{0}_TRANSCRIPT.csv'.format(number), sep='\t').fillna('')

    wavefile = wave.open('{0}_AUDIO.wav'.format(number, 'r'))
    framerate = wavefile.getframerate()
    nframes = wavefile.getnframes()
    wave_data = np.frombuffer(wavefile.readframes(nframes), dtype=np.short)
    reduced_noise = nr.reduce_noise(y=wave_data, sr=framerate)

    responses = []
    response = ''
    response_flag = False
    signal = []

    global counter_train

    for row in transcript.itertuples():
        if row.speaker == 'Ellie':
            content = row.value.strip()
            if identify_topics(content):
                response_flag = True
                if len(response) != 0:
                    response = expanding_contractions(response.strip())
                    response = re.sub(r'[^\w\s]', '', response)
                    responses.append(response)
                response = ''
            elif response_flag and len(content.split()) > 4:
                response_flag = False
                if len(response) != 0:
                    response = expanding_contractions(response)
                    response = re.sub(r'[^\w\s]', '', response)
                    responses.append(response)
                response = ''
        elif row.speaker == 'Participant':
            if 'scrubbed_entry' in row.value:
                continue
            elif response_flag:
                response +=' ' +row.value.split('\n')[0].strip()
            start_time = int(row.start_time*framerate)
            stop_time = int(row.stop_time*framerate)
            signal = np.hstack((signal, reduced_noise[start_time:stop_time].astype(np.float64)))
    # print(responses)
    clip = framerate*15
    if len(responses) == 0:
      print("Empty")
    else:
      # if target == 1 and mode == 'train':
      if target >= 10 and mode == 'train':
        times = 3 if counter_train < 48 else 2
        for i in range(times):
              if clip*(i+1) > len(signal) or 10*(i+1) > len(responses):
                continue
              melspec = librosa.feature.melspectrogram(y=signal[clip*i:clip*(i+1)], n_mels=80,sr=framerate)
              text_f = responses[i*10:(i+1)*10]
              text_features.append(text_f)
              audio_features.append(melspec)
              targets.append(target)
              counter_train+=1
      else:
          melspec = librosa.feature.melspectrogram(y=signal[:clip], n_mels=80, sr=framerate)
          text_f = responses[:10]
          text_features.append(text_f)
          audio_features.append(melspec)
          targets.append(target)

In [21]:
# training set
text_features = []
audio_features = []
train_targets = []

text_features_test = []
audio_features_test = []
test_targets = []
counter_train = 0

In [22]:
for i in range(len(train_split_num)):
    extract_features(train_split_num[i], text_features, audio_features, train_split_clabel[i], train_targets, 'train')

print(np.shape(train_targets))

303
304
305
310
312
313
315
316
317
318
319
320
321
322
324
325
326
327
328
330
333
336
338
339
340
341
343
344
345
347
348
350
351
352
353
355
356
357
358
360
362
363
364
366
368
369
370
371
372
374
375
376
379
380
383
385
386
391
392
393
397
400
401
402
409
412
414
415
416
419
423
425
426
427
428
429
430
433
434
437
441
443
444
445
446
447
448
449
454
455
456
457
459
463
464
468
471
473
474
475
478
479
485
486
487
488
491
(154,)


In [23]:
text_features_test = []
audio_features_test = []
test_targets = []

for i in range(len(test_split_num)):
    extract_features(test_split_num[i], text_features_test, audio_features_test, test_split_clabel[i], test_targets, 'test')

print(np.shape(test_targets))

302
307
331
335
346
367
377
381
382
388
389
390
395
403
404
406
413
417
418
420
422
436
439
440
451
Empty
458
Empty
472
476
477
482
483
484
489
490
492
(33,)


In [194]:
np.savez('fuse_train_text_samples_clas_lab3.npz', text_features)
np.savez('fuse_train_audio_samples_clas_lab3.npz', audio_features)
np.savez('fuse_test_text_samples_clas_lab3.npz', text_features_test)
np.savez('fuse_test_audio_samples_clas_lab3.npz', audio_features_test)
np.savez('fuse_train_labels_clas_lab3.npz', train_targets)
np.savez('fuse_test_labels_clas_lab3.npz', test_targets)


In [24]:
np.savez('fuse_train_text_samples_reg_lab3.npz', text_features)
np.savez('fuse_train_audio_samples_reg_lab3.npz', audio_features)
np.savez('fuse_test_text_samples_reg_lab3.npz', text_features_test)
np.savez('fuse_test_audio_samples_reg_lab3.npz', audio_features_test)
np.savez('fuse_train_labels_reg_lab3.npz', train_targets)
np.savez('fuse_test_labels_reg_lab3.npz', test_targets)

In [42]:
text_features_train = np.load('fuse_train_text_samples_clas_lab3.npz', allow_pickle=True)['arr_0']
audio_features_train = np.load('fuse_train_audio_samples_clas_lab3.npz', allow_pickle=True)['arr_0']
text_features_test = np.load('fuse_test_text_samples_clas_lab3.npz', allow_pickle=True)['arr_0']
audio_features_test = np.load('fuse_test_audio_samples_clas_lab3.npz', allow_pickle=True)['arr_0']
targets_train = np.load('fuse_train_labels_clas_lab3.npz', allow_pickle=True)['arr_0']
ctargets_test = np.load('fuse_test_labels_clas_lab3.npz', allow_pickle=True)['arr_0']

In [196]:
print(text_features_train.shape)
print(audio_features_train.shape)
print(text_features_test.shape)
print(audio_features_test.shape)
print(targets_train.shape)
print(ctargets_test.shape)

(153, 10)
(153, 80, 469)
(33, 10)
(33, 80, 469)
(153,)
(33,)


In [43]:
audio_features_train = np.array([(X - X.min()) / (X.max() - X.min()) for X in audio_features_train])
audio_features_test = np.array([(X - X.min()) / (X.max() - X.min()) for X in audio_features_test])

In [44]:
X_train_text = [' '.join(sublist) for sublist in text_features_train]
X_test_text = [' '.join(sublist) for sublist in text_features_test]
np.shape(X_train_text)

(153,)

In [45]:
train_y = to_categorical(targets_train)
test_y = to_categorical(ctargets_test)

In [46]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train_text)
sequences = tok.texts_to_sequences(X_train_text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [47]:
test_sequences = tok.texts_to_sequences(X_test_text)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

### Classification

In [48]:
input1 = Input(shape=(80,469))
x1 = Conv1D(32, 3, activation='relu')(input1)
x1 = MaxPooling1D(pool_size=4)(x1)
x1 = LSTM(units=64, return_sequences=True)(x1)
x1 = Dropout(0.2)(x1)
x1 = LSTM(units=64, return_sequences=False)(x1)
x1 = Dropout(0.2)(x1)
x1 = Dense(units=32, activation='relu')(x1)

input2 = Input(shape=[max_len])
x2 = Embedding(max_words,100,input_length=max_len)(input2)
x2 = GRU(64, return_sequences=True)(x2)
x2 = Dropout(0.3)(x2)
x2 = GRU(128, return_sequences=False)(x2)
x2 = Dropout(0.3)(x2)

merged = Concatenate(axis=1)([x1, x2])
outputs = Dense(2, activation='softmax')(merged)


model = Model(inputs=[input1, input2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
model.fit([audio_features_train, sequences_matrix], train_y, epochs=10, batch_size=4)


Epoch 1/10
39/39 [==============================] - 21s 216ms/step - loss: 0.6935 - accuracy: 0.4967
Epoch 2/10
39/39 [==============================] - 8s 200ms/step - loss: 0.6928 - accuracy: 0.5229
Epoch 3/10
39/39 [==============================] - 7s 187ms/step - loss: 0.6897 - accuracy: 0.6013
Epoch 4/10
39/39 [==============================] - 9s 230ms/step - loss: 0.6863 - accuracy: 0.7255
Epoch 5/10
39/39 [==============================] - 7s 191ms/step - loss: 0.6785 - accuracy: 0.8562
Epoch 6/10
39/39 [==============================] - 9s 223ms/step - loss: 0.6449 - accuracy: 0.7647
Epoch 7/10
39/39 [==============================] - 7s 188ms/step - loss: 0.4662 - accuracy: 0.8497
Epoch 8/10
39/39 [==============================] - 9s 235ms/step - loss: 0.2463 - accuracy: 0.9020
Epoch 9/10
39/39 [==============================] - 7s 180ms/step - loss: 0.1441 - accuracy: 0.9477
Epoch 10/10
39/39 [==============================] - 10s 260ms/step - loss: 0.0758 - accuracy: 0.97

In [16]:
y_pred = model.predict([audio_features_test, test_sequences_matrix])
predicted_1 = [1 if x[1] > x[0] else 0 for x in y_pred]
print(classification_report(ctargets_test, predicted_1))

2/2 [==============================] - 2s 57ms/step
              precision    recall  f1-score   support

           0       0.89      0.81      0.85        21
           1       0.71      0.83      0.77        12

    accuracy                           0.82        33
   macro avg       0.80      0.82      0.81        33
weighted avg       0.83      0.82      0.82        33



### Regression

In [25]:
text_features_train = np.load('fuse_train_text_samples_reg_lab3.npz', allow_pickle=True)['arr_0']
audio_features_train = np.load('fuse_train_audio_samples_reg_lab3.npz', allow_pickle=True)['arr_0']
text_features_test = np.load('fuse_test_text_samples_reg_lab3.npz', allow_pickle=True)['arr_0']
audio_features_test = np.load('fuse_test_audio_samples_reg_lab3.npz', allow_pickle=True)['arr_0']
targets_train = np.load('fuse_train_labels_reg_lab3.npz', allow_pickle=True)['arr_0']
ctargets_test = np.load('fuse_test_labels_reg_lab3.npz', allow_pickle=True)['arr_0']

In [26]:
audio_features_train = np.array([(X - X.min()) / (X.max() - X.min()) for X in audio_features_train])
audio_features_test = np.array([(X - X.min()) / (X.max() - X.min()) for X in audio_features_test])

In [27]:
X_train_text = [' '.join(sublist) for sublist in text_features_train]
X_test_text = [' '.join(sublist) for sublist in text_features_test]
np.shape(X_train_text)

(154,)

In [31]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train_text)
sequences = tok.texts_to_sequences(X_train_text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [32]:
test_sequences = tok.texts_to_sequences(X_test_text)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [40]:
input1 = Input(shape=(80,469))
x1 = Conv1D(32, 3, activation='relu')(input1)
x1 = MaxPooling1D(pool_size=4)(x1)
x1 = LSTM(units=64, return_sequences=True)(x1)
x1 = Dropout(0.2)(x1)
x1 = LSTM(units=64, return_sequences=False)(x1)
x1 = Dropout(0.2)(x1)
x1 = Dense(units=32, activation='relu')(x1)

input2 = Input(shape=[max_len])
x2 = Embedding(max_words,100,input_length=max_len)(input2)
x2 = GRU(64, return_sequences=True)(x2)
x2 = Dropout(0.3)(x2)
x2 = GRU(128, return_sequences=False)(x2)
x2 = Dropout(0.3)(x2)

merged = Concatenate(axis=1)([x1, x2])
outputs = Dense(1, activation='linear')(merged)


model = Model(inputs=[input1, input2], outputs=outputs)
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0001), metrics=['mean_absolute_error'])

model.fit([audio_features_train, sequences_matrix], targets_train, validation_data=([audio_features_test, test_sequences_matrix], ctargets_test), epochs=15, batch_size=4)

Epoch 1/15
39/39 [==============================] - 23s 285ms/step - loss: 105.5893 - mean_absolute_error: 8.4700 - val_loss: 87.7878 - val_mean_absolute_error: 7.1472
Epoch 2/15
39/39 [==============================] - 7s 178ms/step - loss: 96.5105 - mean_absolute_error: 8.0422 - val_loss: 67.7537 - val_mean_absolute_error: 6.1026
Epoch 3/15
39/39 [==============================] - 9s 228ms/step - loss: 47.1310 - mean_absolute_error: 5.4299 - val_loss: 43.2411 - val_mean_absolute_error: 5.8354
Epoch 4/15
39/39 [==============================] - 6s 163ms/step - loss: 34.9224 - mean_absolute_error: 4.9210 - val_loss: 41.2182 - val_mean_absolute_error: 5.6978
Epoch 5/15
39/39 [==============================] - 9s 229ms/step - loss: 34.3729 - mean_absolute_error: 4.8630 - val_loss: 39.8474 - val_mean_absolute_error: 5.6255
Epoch 6/15
39/39 [==============================] - 6s 164ms/step - loss: 34.8057 - mean_absolute_error: 4.9675 - val_loss: 38.5360 - val_mean_absolute_error: 5.5401
Ep

In [41]:
loss, mean_absolute_error = model.evaluate([audio_features_test, test_sequences_matrix], ctargets_test)
print(f"Test Loss: {loss}, Test Mean Absolute Error: {mean_absolute_error}")
y_pred = model.predict([audio_features_test, test_sequences_matrix])
print(np.sqrt(mean_squared_error(ctargets_test,y_pred)))

2/2 [==============================] - 0s 35ms/step - loss: 33.2129 - mean_absolute_error: 4.5829
Test Loss: 33.2129020690918, Test Mean Absolute Error: 4.582897663116455


2/2 [==============================] - 2s 26ms/step
5.7630634500363636
